In [ ]:
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 25.7 MB/s eta 0:00:00


In [ ]:
!pip install pywebio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.9/500.9 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pywebio: filename=pywebio-1.8.3-py3-none-any.whl size=509502 sha256=86da2cbbb8e5589157e99e539947c4cc4f69a7527b3f390e2ad41bf116764ccf
  Stored in directory: /root/.cache/pip/wheels/fa/72/9f/d0b62ad82e222e5ea96c1fdf07fb86a3ea667da6190a1aca16
Successfully built pywebio


In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import speech_recognition as sr
from pywebio import start_server
from pywebio.input import input, textarea
from pywebio.output import put_text, put_html, put_buttons
from pywebio.session import defer_call
import pickle
import pandas as pd
import re
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import numpy as np

In [ ]:
# Define the preprocess_and_tokenize function
def preprocess_and_tokenize(data):
    data = re.sub("(<.*?>)", "", data)  # Remove html markup
    data = re.sub(r'http\S+', '', data)  # Remove urls
    data = re.sub(r"(#[\d\w\.]+)", '', data)  # Remove hashtags
    data = re.sub(r"(@[\d\w\.]+)", '', data)  # Remove @names
    data = re.sub("(\\W|\\d)", " ", data)  # Remove punctuation and non-ascii digits
    data = data.strip()  # Remove whitespace
    data = word_tokenize(data)  # Tokenization with nltk
    porter = PorterStemmer()  # Stemming with nltk
    stem_data = [porter.stem(word) for word in data]
    return stem_data

In [ ]:
# Load TF-IDF SVM model and TF-IDF vectorizer
tfidf_svm_model = pickle.load(open('/content/drive/MyDrive/NLP_PROJECT/models/tfidf_svm.sav', 'rb'))
df_train = pd.read_csv('/content/drive/MyDrive/NLP_PROJECT/Train.csv')
vectorizer = TfidfVectorizer(tokenizer=preprocess_and_tokenize, sublinear_tf=True, norm='l2', ngram_range=(1, 2))
vectorizer.fit(df_train['Text'].tolist())

<ipython-input-11-1ee0ac1ea69c>:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tfidf_svm_model = pickle.load(open('/content/drive/MyDrive/NLP_PROJECT/models/tfidf_svm.sav', 'rb'))
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limit

TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True,
                tokenizer=<function preprocess_and_tokenize at 0x797e9c401360>)

In [ ]:
# Load CNN model and tokenizer
cnn_model = load_model('/content/drive/MyDrive/NLP_PROJECT/models/cnn_w2v.h5')
tokenizer_cnn = Tokenizer()
tokenizer_cnn.fit_on_texts(df_train['Text'].tolist())

In [ ]:
# Load LSTM model and tokenizer
lstm_model = load_model('/content/drive/MyDrive/NLP_PROJECT/models/biLSTM_w2v.h5')
tokenizer_lstm = Tokenizer()
tokenizer_lstm.fit_on_texts(df_train['Text'].tolist())

In [ ]:
# Define emotion labels
class_names = ['joy', 'sadness', 'fear', 'anger', 'surprise', 'neutral', 'disgust', 'shame', 'guilt']

In [ ]:
def handle_input(user_input):
    put_text("User:", user_input)

    # Preprocess user input for TF-IDF SVM model
    user_input_tfidf = ' '.join(preprocess_and_tokenize(user_input))
    # Preprocess user input for CNN model
    sequence_cnn = tokenizer_cnn.texts_to_sequences([user_input])
    padded_sequence_cnn = pad_sequences(sequence_cnn, maxlen=500)
    # Preprocess user input for LSTM model
    sequence_lstm = tokenizer_lstm.texts_to_sequences([user_input])
    padded_sequence_lstm = pad_sequences(sequence_lstm, maxlen=500)

    # Predictions
    prediction_tfidf = tfidf_svm_model.predict([user_input_tfidf])[0]
    prediction_cnn = cnn_model.predict(padded_sequence_cnn)[0]
    prediction_lstm = lstm_model.predict(padded_sequence_lstm)[0]

    # Combine predictions using weighted average
    combined_pred = (0.7 * prediction_tfidf + 0.756 * prediction_cnn + 0.66 * prediction_lstm) / (0.7 + 0.756 + 0.66)

    # Get the predicted label
    predicted_label_index = np.argmax(combined_pred)
    predicted_label = class_names[predicted_label_index]

    put_text("Combined Prediction:", predicted_label)


In [ ]:
# Global variable to track the state of recording
is_recording = False

# Function to handle speech input
def handle_speech_input():
    global is_recording

    if not is_recording:
        start_recording()
    else:
        stop_recording()

def start_recording():
    global is_recording
    is_recording = True

    recognizer = sr.Recognizer()

    # Display the microphone prompt
    put_text("Speak", "Speak something...")

    with sr.Microphone() as source:
        audio = recognizer.listen(source)
        put_text("Processing...")

    try:
        user_input = recognizer.recognize_google(audio)
        handle_input(user_input)
    except sr.UnknownValueError:
        put_text("Sorry, I could not understand what you said.")
    except sr.RequestError as e:
        put_text("Sorry, I could not request results from Google Speech Recognition service; {0}".format(e))

    is_recording = False

# Function to stop recording
def stop_recording():
    global is_recording
    is_recording = False

In [ ]:
# Function to define the chatbot app
def chatbot_app():
    put_html("<div style='background-color: white; padding: 20px;'> <h1>Emotion Depicter</h1> </div>")
    put_buttons(['Please Write', '🎤'], onclick=[handle_text_input, handle_speech_input])

In [ ]:
# Start the server
if __name__ == "__main__":
    start_server(chatbot_app, port=8086)

Running on all addresses.
Use http://172.28.0.12:43395/ to access the application


RuntimeError: This event loop is already running